In [ ]:
import csv
from pathlib import Path 
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn.utils import shuffle
from keras.preprocessing import sequence
import os 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, Reshape
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


In [0]:
max_features = 20000
maxlen = 100

Ytrain=[]
trainx=[]
Ytest=[]
testx=[]
print("loading the data...")
with open('out5.csv',errors='ignore') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = '\t')
    for row in csv_reader:
        if csv_reader.line_num < 170001:
            Ytrain.append(int(row[0].split('/')[0]))
            trainx.append(row[1])
        elif csv_reader.line_num < 190001:
            Ytest.append(int(row[0].split('/')[0]))
            testx.append(row[1]) 
        else:
            break    

#194869


print("data loaded, processing data...")


# create the tokenizer
tok= Tokenizer(num_words=max_features, lower=True,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split=' ', char_level=False, oov_token=None, document_count=0)
# fit the tokenizer on the documents
tok.fit_on_texts(trainx)
# summarize what was learned
#print(train.word_counts)
#print(train.document_count)
#print(train.word_index)
#print(train.word_docs)
# integer encode documents
Xtrain = tok.texts_to_sequences(trainx)

Xtest = tok.texts_to_sequences(testx)




#Xtrain, y_train = shuffle(Xtrain, Ytrain, random_state=0)
#Xtest, y_test = shuffle(Xtest, Ytest, random_state=0)


# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Ytrain)
encoded_Ytrain = encoder.transform(Ytrain)
encoded_Ytest = encoder.transform(Ytest)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Ytrain)
y_test = np_utils.to_categorical(encoded_Ytest)

encoder_length = len(y_train[0])
print("length of one hot enconding  ", encoder_length)

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(Xtrain, maxlen=maxlen)
x_test = sequence.pad_sequences(Xtest, maxlen=maxlen)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

loading the data...
data loaded, processing data...


length of one hot enconding   10
Pad sequences (samples x time)
x_train shape: (169990, 100)
x_test shape: (20000, 100)


In [0]:
#https://medium.com/datadriveninvestor/deep-learning-techniques-for-text-classification-9392ca9492c7


from keras.callbacks import EarlyStopping
from keras.layers import GRU

embedding_size = 128
gru_node = 128
dropout = 0.5


model = Sequential()
model.add(Embedding(max_features,embedding_size, input_length=maxlen))

model.add(GRU(gru_node,return_sequences=True, recurrent_dropout=0.2))
model.add(Dropout(dropout))
model.add(GRU(gru_node, recurrent_dropout=0.2))
model.add(Dropout(dropout))
model.add(Dense(32, activation='relu'))
model.add(Dense(encoder_length, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 
batch_size = 64


history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)



/opt/venv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
Train on 152991 samples, validate on 16999 samples
Epoch 1/3
 65792/152991 [===========>..................] - ETA: 43:02 - loss: 2.0919 - accuracy: 0.2986

 84992/152991 [===============>..............] - ETA: 32:55 - loss: 2.0210 - accuracy: 0.3111

118400/152991 [======================>.......] - ETA: 16:11 - loss: 1.9321 - accuracy: 0.3302

125312/152991 [=======================>......] - ETA: 12:57 - loss: 1.9179 - accuracy: 0.3334

142080/152991 [==========================>...] - ETA: 5:03 - loss: 1.8885 - accuracy: 0.3404

152991/152991 [==============================] - 4321s 28ms/step - loss: 1.8717 - accuracy: 0.3439 - val_loss: 1.8151 - val_accuracy: 0.3467
Epoch 2/3
 14464/152991 [=>............................] - ETA: 55:33 - loss: 1.5511 - accuracy: 0.4372

152991/152991 [==============================] - 3608s 24ms/step - loss: 9.6244 - accuracy: 0.4008 - val_loss: 1.8349 - val_accuracy: 0.3406
Epoch 3/3
20000/20000 [==============================] - 60s 3ms/step
Test score: 1.775944815826416
Test accuracy: 0.35315001010894775
